# Settings

In [4]:
import os
import numpy as np

from keras.models import Model
from keras.layers import Dense, Activation,Flatten, Conv2D, MaxPooling2D
from keras.layers import Input, ZeroPadding2D, BatchNormalization, Add, AveragePooling2D
from keras import backend
from keras.initializers import glorot_uniform
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

backend.set_image_data_format('channels_last')
train_path = './Dataset/train'
test_path = './Dataset/test'

# Data Preprocessing

In [5]:
# the augmentation tool for data augmentation
image_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

In [8]:
def augmentation(generator):      
    """
    :param tuple generator: ImageDataGenerator
    :return array: transformed tenfold images
    """
    
    classes_list = os.listdir(train_path)

    for class_name in classes_list:
        img_list = os.listdir(os.path.join(train_path, class_name))

    for img_name in img_list:
            img_path_name = os.path.join(train_path, class_name, img_name)
            img = image.load_img(img_path_name)
            img = image.img_to_array(img)

            img = img.reshape((1,) + img.shape)

            i = 0

            for batch in generator.flow(img, batch_size=1, save_to_dir=os.path.join(train_path, class_name),
                save_prefix=class_name, save_format='jpg'):
                i += 1
                if i > 10:
                    break

augmentation(image_gen)

In [24]:
def load_preprocessed_data(path):
    """
    :param string path: train_path or test_path
    :return array, array: dataset(features), Y(label)
    """
    
    dataset = []
    y = []

    classes_list = os.listdir(path)

    for class_number, class_name in enumerate(classes_list):
        img_list = os.listdir(os.path.join(path, class_name))

        for img_number, img_name in enumerate(img_list):
            img_path_name = os.path.join(path, class_name, img_name)

            img = image.load_img(img_path_name, target_size=(64, 64))

            # img -> np.array
            # shape : (height, width, channels) = (128, 128, 3)
            img_input = image.img_to_array(img)
            dataset.append(img_input)
            y.append(class_number)
    
    dataset = np.array(dataset)
    y = np.array(y)
    Y = np.eye(3)[y.astype(int)]
    dataset = dataset.astype('float64')
    dataset /= 255
    
    return dataset, Y

# Modeling - ResNet50

In [11]:
# identity block

def identity_block(X, kernel, filters, stage, block):
    """
    :param tensor X: input tensor (examples, height_prev, width_prev, channel_prev)
    :param int kernel: kernel size
    :param list filters: filsters of CONV layer
    :param int stage: n-th stage
    :param string block: m-th block in nth stage

    :return tensor: output of the identity block - tensor: (height, width, channel)
    """
    
    # setting
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X

    # 1st component
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base+'2a',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # 2nd compoenent
    X = Conv2D(filters=F2, kernel_size=(kernel, kernel), strides=(1, 1), padding='same', name=conv_name_base+'2b',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # 3rd component
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base+'2c',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # add shorcut again
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [12]:
def conv_block(X, kernel, filters, stage, block, s=2):    
    """
    :param tensor X: input tensor (examples, height_prev, width_prev, channel_prev)
    :param int kernel: kernel size
    :param list filters: filsters of CONV layer
    :param int stage: n-th stage
    :param string block: m-th block in nth stage
    :param int s: strides

    :return tensor: X -- output of the convolutional block, tensor: (height, width, channel)
    """
    
    # setting
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X

    # 1st component
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), name=conv_name_base+'2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)

    # 2nd compoenent
    X = Conv2D(filters=F2, kernel_size=(kernel, kernel), strides=(1, 1), name=conv_name_base+'2b', padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)

    # 3rd component
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), name=conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)

    # apply 1X1 conv to shortcut as well
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), name=conv_name_base+'1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)

    # add shorcut again
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [14]:
class CNN(Model):
    def __init__(self, in_shape=(128, 128, 3), classes=3):
        self.in_shape = in_shape
        self.classes = classes
        self.build_model()
        super().__init__(self.X_input, self.Y)
        self.compile()


    def build_model(self):
        in_shape = self.in_shape
        classes = self.classes
        X_input = Input(shape=in_shape)
        X = ZeroPadding2D((3, 3))(X_input)

        # Stage 1
        X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), name='conv1',
                   kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name='bn_conv1')(X)
        X = Activation('relu')(X)
        X = MaxPooling2D((3, 3), strides=(2, 2))(X)

        # Stage 2
        X = conv_block(X, kernel=3, filters=[64, 64, 256], stage=2, block='a', s=1)
        X = identity_block(X, kernel=3, filters=[64, 64, 256], stage=2, block='b')
        X = identity_block(X, kernel=3, filters=[64, 64, 256], stage=2, block='c')

        # Stage 3
        X = conv_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='a', s=2)
        X = identity_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='b')
        X = identity_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='c')
        X = identity_block(X, kernel=3, filters=[128, 128, 512], stage=3, block='d')

        # Stage 4
        X = conv_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
        X = identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='b')
        X = identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='c')
        X = identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='d')
        X = identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='e')
        X = identity_block(X, kernel=3, filters=[256, 256, 1024], stage=4, block='kernel')

        # Stage 5
        X = conv_block(X, kernel=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
        X = identity_block(X, kernel=3, filters=[512, 512, 2048], stage=5, block='b')
        X = identity_block(X, kernel=3, filters=[512, 512, 2048], stage=5, block='c')

        X = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', name='avg_pool')(X)
        X = Flatten()(X)
        Y = Dense(units=classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

        self.X_input, self.Y = X_input, Y

    def compile(self):
        Model.compile(self, optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train

In [15]:
dataset, Y = load_preprocessed_data(path=train_path)
X_train, X_val, Y_train, Y_val = train_test_split(dataset, Y, test_size=0.1, stratify=Y, random_state=0)

print("X shape: ", X_train.shape)
print("Y shape: ", Y_train.shape)

Resnet = CNN(in_shape=(64, 64, 3), classes=3)
Resnet.fit(X_train, Y_train, batch_size=32, epochs=5, validation_data=(X_val, Y_val), callbacks=[EarlyStopping(patience=10)])


# result : validation accuracy - 100% after 5th epoch

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[[[[ 53.  47.  47.]
   [ 38.  39.  39.]
   [ 36.  39.  42.]
   ...
   [ 64.  71.  79.]
   [ 63.  70.  78.]
   [ 65.  72.  80.]]

  [[ 51.  46.  48.]
   [ 40.  42.  43.]
   [ 37.  42.  45.]
   ...
   [ 64.  71.  79.]
   [ 62.  69.  77.]
   [ 66.  73.  81.]]

  [[ 48.  45.  49.]
   [ 41.  46.  49.]
   [ 39.  46.  50.]
   ...
   [ 63.  70.  78.]
   [ 64.  71.  79.]
   [ 69.  76.  84.]]

  ...

  [[173. 127. 112.]
   [173. 126. 111.]
   [171. 124. 109.]
   ...
   [ 40.  43.  48.]
   [ 41.  44.  49.]
   [ 41.  44.  49.]]

  [[162. 118. 105.]
   [162. 118. 105.]
   [161. 117. 104.]
   ...
   [ 43.  46.  51.]
   [ 41.  44.  49.]
   [ 43.  46.  51.]]

  [[153. 111.  99.]
   [154. 112. 100.]
   [156. 114. 102.]
   ...
   [ 43.  46.  51.]
   [ 42.  45.  50.]
   [ 43.  46.  51.]]]


 [[[ 46.  47.  52.]
   [ 48.  51.  56.]
   [ 48.  52.  56.]
   ...
   [ 61.  69.  72.]
   [ 58.  66.  69.]
   [ 62.  70.  73.]]

  [[ 51.  52.  57.]
   [ 51.  54.  59.]
   [ 51.  56.  60.]
   ...
   [ 61.  69.  72.]
 

[[[[0.20784314 0.18431373 0.18431373]
   [0.14901961 0.15294118 0.15294118]
   [0.14117647 0.15294118 0.16470588]
   ...
   [0.25098039 0.27843137 0.30980392]
   [0.24705882 0.2745098  0.30588235]
   [0.25490196 0.28235294 0.31372549]]

  [[0.2        0.18039216 0.18823529]
   [0.15686275 0.16470588 0.16862745]
   [0.14509804 0.16470588 0.17647059]
   ...
   [0.25098039 0.27843137 0.30980392]
   [0.24313725 0.27058824 0.30196078]
   [0.25882353 0.28627451 0.31764706]]

  [[0.18823529 0.17647059 0.19215686]
   [0.16078431 0.18039216 0.19215686]
   [0.15294118 0.18039216 0.19607843]
   ...
   [0.24705882 0.2745098  0.30588235]
   [0.25098039 0.27843137 0.30980392]
   [0.27058824 0.29803922 0.32941176]]

  ...

  [[0.67843137 0.49803922 0.43921569]
   [0.67843137 0.49411765 0.43529412]
   [0.67058824 0.48627451 0.42745098]
   ...
   [0.15686275 0.16862745 0.18823529]
   [0.16078431 0.17254902 0.19215686]
   [0.16078431 0.17254902 0.19215686]]

  [[0.63529412 0.4627451  0.41176471]
   [0.6

X shape:  (30771, 64, 64, 3)
Y shape:  (30771, 3)





Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 30771 samples, validate on 3420 samples
Epoch 1/5




30771/30771 [==============================] - 2223s - loss: 0.0816 - acc: 0.9795 - val_loss: 0.1634 - val_acc: 0.9520
Epoch 2/5
30771/30771 [==============================] - 2112s - loss: 0.0070 - acc: 0.9985 - val_loss: 0.0182 - val_acc: 0.9953
Epoch 3/5
30771/30771 [==============================] - 2138s - loss: 0.0039 - acc: 0.9989 - val_loss: 1.6288e-05 - val_acc: 1.0000
Epoch 4/5
30771/30771 [==============================] - 2205s - loss: 0.0165 - acc: 0.9971 - val_loss: 0.0450 - val_acc: 0.9930
Epoch 5/5

# Test

In [25]:
X_test, Y_test = load_preprocessed_data(path=test_path)
print("X shape: ", X_test.shape)
print("Y shape: ", Y_test.shape)
Y_predicted = Resnet.predict(X_test, batch_size=32)
P = np.argmax(Y_predicted, axis=1)
T = np.argmax(Y_test, axis=1)
np.sum(P == T) / 70

X shape:  (70, 64, 64, 3)
Y shape:  (70, 3)


1.0